<a href="https://colab.research.google.com/github/Edenshmuel/PapaJohns_Data_Science_Project/blob/main/Temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Final_Project_PapaJohns/All_years.csv')

<ipython-input-3-44878571b353>:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Final_Project_PapaJohns/All_years.csv')


In [4]:
df

,Date,Month,סוג פריט,תאור סוג,מס' פריט,תאור פריט,הזמנה,כמות,סכום,הנחה לפריט,אחרי הנחה,Is_original
0,2022-01-01,1,0.0,NaN,67,NaN,14328.0,1,4.95,0.0,4.95,1
1,2022-01-01,1,0.0,NaN,67,NaN,14354.0,2,9.90,0.0,9.90,1
2,2022-01-01,1,1.0,פיצות מיוחדות 14',4,ביאנקה 14' [],14321.0,1,85.90,0.0,85.90,1
3,2022-01-01,1,1.0,פיצות מיוחדות 14',7,טוסקנית 14' [],14304.0,1,85.90,0.0,85.90,1
4,2022-01-01,1,1.0,פיצות מיוחדות 14',13,המומלצת 14' [],14358.0,1,89.90,0.0,89.90,1
...,...,...,...,...,...,...,...,...,...,...,...,...
258189,2024-12-31,12,52,מנות נלוות,3117,שובר 50 שח DV,7669633.0,5,125.00,0.0,125.00,1
258190,2024-12-31,12,52,מנות נלוות,3210,2 משפחתיות קלאסיות + 2 תוספות צמחוניות חבר,7669582.0,1,0.00,0.0,0.00,1
258191,2024-12-31,12,52,מנות נלוות,3210,2 משפחתיות קלאסיות + 2 תוספות צמחוניות חבר,7669585.0,1,19.90,0.0,19.90,1
258192,2024-12-31,12,52,מנות נלוות,3210,2 משפחתיות קלאסיות + 2 תוספות צמחוניות חבר,7669605.0,1,19.90,0.0,19.90,1


In [5]:
from google.colab import files
uploaded = files.upload()

Saving preprocessing.py to preprocessing.py


In [13]:
!pip install convertdate

In [6]:
# אם הקובץ נקרא preprocessing.py
import preprocessing

# טען את הפונקציה
from preprocessing import prepare_data

In [7]:
df_ready = prepare_data(df)

In [8]:
df_ready.columns

Index(['Date', 'Year', 'Month', 'Day', 'Day_Name', 'Is_Weekend',
       'Description Categories', 'Item Description', 'Quantity',
       'is_christian_holiday', 'is_jewish_holiday', 'is_near_jewish_holiday',
       'is_day_before_new_year', 'clean_desc_encoded', 'category_encoded',
       'WeekOfYear', 'Season', 'is_start_of_month', 'is_end_of_month',
       'Day_Name_sin', 'Day_Name_cos', 'Month_sin', 'Month_cos',
       'avg_quantity_all_time', 'std_quantity_all_time', 'num_days_sold',
       'popularity_score', 'encoded_jewish_holiday',
       'encoded_christian_holiday', 'encoded_portion_type'],
      dtype='object')

In [9]:
df_ready["Item Description"]

,Item Description
0,ביאנקה 14
268,ארוחת שחיתות
267,4 פיצות אישיות
266,פאפא דיל
265,פאפא דיל
...,...
236503,עגבניות
236504,עגבניות
236505,עגבניות
236497,זיתים שחורים


In [ ]:
!grep -R "כמות" preprocessing.py
!grep -R "agg" preprocessing.py


    df = df[df["כמות"] >= 0]
        'תאור פריט','הזמנה','כמות','סכום',
        base_data = row.drop(["כמות בפועל", "תאור פריט", "כמות"], errors="ignore").to_dict()
        quantity_dict = row.get("כמות בפועל", {})
        default_qty = row["כמות"]
    df["כמות בפועל"] = df["תאור פריט"].apply(extract_quantity_by_keywords)
    agg_features = df.groupby("clean_desc_encoded").agg({
    agg_features.columns = ["avg_quantity_all_time", "std_quantity_all_time", "num_days_sold"]
    agg_features = agg_features.reset_index()
    agg_features["popularity_score"] = agg_features["avg_quantity_all_time"]
    df = df.merge(agg_features, on="clean_desc_encoded", how="left")


In [ ]:
!sed -n '350,380p' preprocessing.py



def sort_by_date(df):
    df = df.copy()
    df["Date"] = pd.to_datetime(df["Date"])
    
    return df.sort_values("Date")

def prepare_data(df):
    df = drop(df)
    df = convert_date(df)
    df = split_category(df)
    df["כמות בפועל"] = df["תאור פריט"].apply(extract_quantity_by_keywords)
    df = split_rows_by_category_and_quantity(df)
    df = add_holiday_features(df)
    df = encode_features(df)
    df = add_time_features(df)
    df = add_product_features(df)
    df = encode_holiday_and_portion(df)
    df = clean_final_columns(df)
    df = sort_by_date(df)
    
    return df



  

